# 스타벅스와 이디야 

- 자료 위치 : 소상공인시장진흥공단_상가업소정보_201906
- https://www.data.go.kr/dataset/15012005/fileData.do

- 뉴스기사 : 스타벅스 '쏠림' vs 이디야 '분산'
- http://news.bizwatch.co.kr/article/consumer/2018/01/19/0015

# 환경설정

In [5]:
import pandas as pd
import numpy as np
import re

# 경고 메세지 없애기
import warnings
warnings.filterwarnings('ignore')

# 시각화 라이브러리 임포트 
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.font_manager as fm
path = 'C:\Windows\Fonts\malgunbd.ttf'
font_name = fm.FontProperties(fname=path).get_name()
print(font_name)
plt.rc('font', family=font_name)

import seaborn as sns

# 지도 표현
import folium

Malgun Gothic


In [6]:
shop_2019 = pd.read_csv('data/소상공인시장진흥공단_상가업소정보_201906_01.csv')
shop_2019.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,19911397,커피빈코리아선릉로93길점,코리아선릉로93길점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1168010100106960042022041,NaN,서울특별시 강남구 선릉로93길 6,135080,6149.0,NaN,1,NaN,127.047883,37.505675
1,19911801,프로포즈,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,...,1154510200101620001017748,NaN,서울특별시 금천구 가산로 34-6,153010,8545.0,NaN,1,NaN,126.899220,37.471711
2,19912201,싱싱커피&토스트,NaN,Q,음식,Q07,패스트푸드,Q07A10,토스트전문,I56192,...,2653010400105780000002037,산업용품유통상가,부산광역시 사상구 괘감로 37,617726,46977.0,NaN,1,26,128.980455,35.159774
3,19905471,와라와라호프,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,...,1168010600106040000014378,대치상가,서울특별시 강남구 남부순환로 2933,135280,6280.0,NaN,1,NaN,127.061026,37.493922
4,19932756,가락사우나내스낵,NaN,F,생활서비스,F09,대중목욕탕/휴게,F09A02,사우나/증기탕/온천,S96121,...,1171010500102560005010490,NaN,서울특별시 송파구 가락로 71,138846,5690.0,NaN,1,NaN,127.104071,37.500249


In [ ]:
shop_2019.shape

In [ ]:
shop_2019.columns

In [10]:
df_seoul = shop_2019.loc[shop_2019['시도명'].str.startswith('서울')].copy()
df_seoul.shape

(401071, 39)

## 결측치 확인하기

In [11]:
df_seoul.isnull().sum()

상가업소번호            0
상호명               0
지점명          344270
상권업종대분류코드         0
상권업종대분류명          0
상권업종중분류코드         0
상권업종중분류명          0
상권업종소분류코드         0
상권업종소분류명          0
표준산업분류코드      23587
표준산업분류명       23587
시도코드              0
시도명               0
시군구코드             0
시군구명              0
행정동코드             0
행정동명              0
법정동코드             0
법정동명              0
지번코드              0
대지구분코드            0
대지구분명             0
지번본번지             0
지번부번지         75924
지번주소              0
도로명코드             0
도로명               0
건물본번지             0
건물부번지        354031
건물관리번호            0
건물명          205736
도로명주소             0
구우편번호             0
신우편번호             2
동정보          369770
층정보          125634
호정보          343797
경도                0
위도                0
dtype: int64

## 서울에 있는 스타벅스, 이디야 데이타프레임 생성하기 

In [12]:
df_seoul.loc[df_seoul['상호명'].str.contains('스타벅스|starbucks|STARBUCKS'), '상호명'].shape

(502,)

In [13]:
df_seoul.loc[df_seoul['상호명'].str.contains('이디야|ediya|EDIYA'), '상호명'].shape

(501,)

In [14]:
df_cafe = df_seoul[df_seoul['상호명'].str.contains('스타벅스|starbucks|STARBUCKS|이디야|ediya|EDIYA')]
df_cafe.shape

(1003, 39)

In [15]:
df_cafe.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
1219,19976038,스타벅스,이태원점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1117013000101270029009542,NaN,서울특별시 용산구 이태원로 188,140200,4406.0,NaN,NaN,29,126.994781,37.534303
1676,25435527,이디야커피,신길역점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1156013200100590002019075,엔씨티오피스텔,서울특별시 영등포구 영등포로 353,150050,7318.0,NaN,1,NaN,126.918062,37.515118
1946,19998811,스타벅스종로3가점,종로3가점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1111015600100090003016323,NaN,서울특별시 종로구 종로 113-1,110123,3139.0,NaN,1,NaN,126.990207,37.570585
2840,20680373,스타벅스,신사2점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1165010600100280001019301,푸른빌딩,서울특별시 서초구 강남대로 581,137904,6530.0,NaN,2,NaN,127.019763,37.513663
2886,20592419,이디야커피,이마트구로점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1153010200101880026022963,이마트구로점,서울특별시 구로구 디지털로32길 43,152050,8379.0,NaN,3,NaN,126.897870,37.484385


In [16]:
df_seoul.loc[df_seoul['상호명'].str.contains('스타벅스|starbucks|STARBUCKS'), '상호명'].value_counts()

스타벅스                296
스타벅스양천향교역             2
스타벅스동교삼거리점            1
스타벅스가든파이브             1
스타벅스논현힐탑              1
스타벅스종각점               1
스타벅스구파발역점             1
스타벅스명동역점              1
스타벅스마포아크로타워점          1
스타벅스송파구청점             1
스타벅스올림픽공원북문점          1
스타벅스이수역사거리점           1
스타벅스성신여대정문            1
스타벅스W-MALL점           1
스타벅스커피여의도호성           1
스타벅스화곡DT점             1
스타벅스천호로데오점            1
스타벅스화곡동점              1
스타벅스가산디지털단지점          1
스타벅스송파방이DT점           1
STARBUCKSCOFFEE       1
스타벅스구산역점              1
스타벅스역점                1
스타벅스사당점               1
스타벅스동여의도점             1
스타벅스광운대점              1
스타벅스서울교대점             1
스타벅스신천역점              1
스타벅스새문안로점             1
스타벅스서울대역점             1
                   ... 
스타벅스남산스테이트            1
스타벅스서울대입구역점           1
스타벅스선릉로점              1
스타벅스역삼럭키점             1
스타벅스광장점               1
스타벅스명지대점              1
스타벅스예술의전당점            1
스타벅스동교점               1
스타벅스학여울점              1
스타벅스신촌명물거리점           1
스타벅스중구저동점       

In [17]:
df_cafe.loc[df_cafe['상호명'].str.contains('스타벅스|starbucks|STARBUCKS'), '브랜드명'] = '스타벅스'

In [18]:
df_cafe[['상호명', '브랜드명']].head(20)

,상호명,브랜드명
1219,스타벅스,스타벅스
1676,이디야커피,NaN
1946,스타벅스종로3가점,스타벅스
2840,스타벅스,스타벅스
2886,이디야커피,NaN
8718,이디야커피,NaN
14596,이디야커피,NaN
14939,스타벅스커피여의도IFC1F,스타벅스
14951,스타벅스,스타벅스
15161,이디야커피,NaN


### '브랜드명'이 NaN인 결측치 값을 '이디야'로 채우기

In [19]:
df_cafe.loc[df_cafe['브랜드명'].isnull(), '상호명'].head()

1676     이디야커피
2886     이디야커피
8718     이디야커피
14596    이디야커피
15161    이디야커피
Name: 상호명, dtype: object

In [20]:
df_cafe['브랜드명'] = df_cafe['브랜드명'].fillna('이디야')

In [21]:
df_cafe['브랜드명'].value_counts()

스타벅스    502
이디야     501
Name: 브랜드명, dtype: int64

## '상호명','브랜드명' 컬럼 확인하기 

In [22]:
df_cafe[['상호명','브랜드명']].head()

,상호명,브랜드명
1219,스타벅스,스타벅스
1676,이디야커피,이디야
1946,스타벅스종로3가점,스타벅스
2840,스타벅스,스타벅스
2886,이디야커피,이디야


In [23]:
df_cafe['브랜드명'].value_counts()

스타벅스    502
이디야     501
Name: 브랜드명, dtype: int64

## folium 시각화

In [24]:
geo_df = df_cafe

In [27]:
map = folium.Map(location=[geo_df['위도'].mean(), 
                           geo_df['경도'].mean()], 
                 zoom_start=12, 
                 tiles='Stamen Toner')

for n in geo_df.index:    
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    
    if geo_df.loc[n, '브랜드명'] == '스타벅스' :
        icon_color = 'green'
    else:
        icon_color = 'blue'  
    
    folium.CircleMarker(
        location=[geo_df.loc[n,'위도'], 
                  geo_df.loc[n, '경도']],
        radius=3,
        popup=popup_name,
        color= icon_color,
        fill=True,
        fill_color=icon_color
    ).add_to(map)


map

AttributeError: module 'folium' has no attribute 'Map'

## 구별 브랜드별 점포수로 데이타프레임 생성하기

In [ ]:
df_cafe.groupby(['시군구명', '브랜드명'])['상호명'].count()

In [ ]:
df_cafe_brand = pd.DataFrame(
    df_cafe.groupby(['시군구명', '브랜드명'])['상호명'].count()
).reset_index()

In [ ]:
df_cafe_brand

In [ ]:
df_cafe_brand.columns = ['구', '브랜드명', '매장수']
df_cafe_brand

## 시각화

In [ ]:
plt.figure(figsize=(24, 6))
sns.barplot(data=df_cafe_brand, x='구', y='매장수', 
            hue='브랜드명')

## groupby로 구별로 갯수 확인하기

In [ ]:
df_cafe.groupby(['시군구명'])['상호명'].count()

In [ ]:
df_cafe_gu = pd.DataFrame(
    df_cafe.groupby(['시군구명'])['상호명'].count()
 )
df_cafe_gu.columns = ['매장수']
df_cafe_gu.head()

## choropleth 그리기

* 구별로 매장수를 표현

In [ ]:
import json
geo_path = '../data/seoul_municipalities_geo_simple.json'
geo_json = json.load(open(geo_path, encoding='utf-8'))
geo_json

### 스타벅스 이디야 카페 매장 합계 분포

In [ ]:


map = folium.Map(location=[geo_df['위도'].mean(), 
                           geo_df['경도'].mean()], 
                 zoom_start=12)

map.choropleth(geo_data = geo_json,
data = df_cafe_gu['매장수'],
columns = [df_cafe_gu.index, df_cafe_gu['매장수']],
fill_color = 'Purples',
key_on = 'properties.SIG_KOR_NM',
fill_opacity=0.7,
line_opacity=0.2,
highlight=True)


for n in geo_df.index:
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    
    if geo_df.loc[n, '브랜드명'] == '스타벅스' :
        icon_color = 'green'
    else:
        icon_color = 'blue'    
    
    # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
    # --> folium.vector_layer.CircleMarker 를 사용합니다. 
    folium.CircleMarker(
        location=[geo_df.loc[n,'위도'], 
                  geo_df.loc[n, '경도']],
        radius=3,
        popup=popup_name,
        color= icon_color,
        fill=True,
        fill_color=icon_color
    ).add_to(map)

map

### 스타벅스 매장 분포

In [9]:
df_cafe_temp = df_cafe[df_cafe['브랜드명'] == '스타벅스']
df_cafe_starbucks = pd.DataFrame(
    df_cafe_temp.groupby(['시군구명'])['상호명'].count()
)
df_cafe_starbucks.columns = ['매장수']
df_cafe_starbucks.head()

NameError: name 'df_cafe' is not defined

In [ ]:
map = folium.Map(location=[geo_df['위도'].mean(), 
                           geo_df['경도'].mean()], 
                 zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_data = geo_json,
               data = df_cafe_starbucks['매장수'],
               columns = [df_cafe_starbucks.index, 
                          df_cafe_starbucks['매장수']],
               fill_color = 'YlGn',
               key_on = 'properties.SIG_KOR_NM',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for n in geo_df.index:
    
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    
    if geo_df.loc[n, '브랜드명'] == '스타벅스' :
        icon_color = 'green'    

        folium.CircleMarker(
            location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
            radius=3,
            popup=popup_name,
            color= icon_color,
            fill=True,
            fill_color=icon_color
        ).add_to(map)


map

### 이디야 매장 분포

In [ ]:
df_cafe_temp = df_cafe[df_cafe['브랜드명'] == '이디야']
df_cafe_ediya = pd.DataFrame(df_cafe_temp.groupby(['시군구명'])['상호명'].count())
df_cafe_ediya.columns = ['매장수']
df_cafe_ediya.head()

In [ ]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_data = geo_json,
               data = df_cafe_ediya['매장수'],
               columns = [df_cafe_ediya.index, df_cafe_ediya['매장수']],
               fill_color = 'PuBu',
               key_on = 'properties.SIG_KOR_NM',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for n in geo_df.index:
    
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    
    if geo_df.loc[n, '브랜드명'] == '이디야' :
        icon_color = 'blue'    

        folium.CircleMarker(
            location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
            radius=3,
            popup=popup_name,
            color= icon_color,
            fill=True,
            fill_color=icon_color
        ).add_to(map)


map

# 스타벅스와 이디야 비교

In [ ]:
df_cafe_brand.head()
df_cafe_brand.columns = ['구', '브랜드명','매장수']

In [ ]:
df_cafe_brand.head()

In [ ]:
df_cafe_brand_vs = df_cafe_brand.pivot_table(index='구',columns='브랜드명',values='매장수')
df_cafe_brand_vs.columns = ['스타벅스', '이디야']
df_cafe_brand_vs.head()

In [ ]:
df_cafe_brand_vs['매장수비교'] = df_cafe_brand_vs.apply(lambda x : 1 if x['스타벅스'] > x['이디야'] else 0, axis=1)
df_cafe_brand_vs.head()

In [ ]:
lng_list = []
lat_list = []
for gu in df_cafe_brand_vs.index:
    lat = df_cafe.loc[df_cafe['시군구명'] == gu, '위도'].mean()
    lng = df_cafe.loc[df_cafe['시군구명'] == gu, '경도'].mean()
    lat_list.append(lat)
    lng_list.append(lng)
    
df_cafe_brand_vs['위도'] = lat_list
df_cafe_brand_vs['경도'] = lng_list
df_cafe_brand_vs.head()

In [ ]:
df_cafe_brand_vs['스타벅스'] = df_cafe_brand_vs['스타벅스'].astype(float)
df_cafe_brand_vs['이디야'] = df_cafe_brand_vs['이디야'].astype(float)
df_cafe_brand_vs.info()

In [ ]:
map = folium.Map(location=[geo_df['위도'].mean(), 
                           geo_df['경도'].mean()], 
                 zoom_start=11, 
                 tiles='Stamen Toner')

map.choropleth(geo_data = geo_json,
       data = df_cafe_brand_vs['매장수비교'],
       columns = [df_cafe_brand_vs.index, 
                  df_cafe_brand_vs['매장수비교']],
       fill_color = 'BuGn',
       key_on = 'properties.SIG_KOR_NM',
       fill_opacity=0.7,
       line_opacity=0.2,
       highlight=True)


for gu in df_cafe_brand_vs.index:
        
    for cafe in ['스타벅스', '이디야']:
        
        cafe_count = df_cafe_brand_vs.loc[gu, cafe]
        msg = f'{gu} {cafe} 매장수 : {cafe_count:.0f}'
        
        icon_color = 'blue'
        if cafe == '스타벅스': 
            icon_color = 'green'
        
        folium.CircleMarker(
    location = [df_cafe_brand_vs.loc[gu,'위도'], 
                df_cafe_brand_vs.loc[gu,'경도']],
    radius = cafe_count,
    color = icon_color,
    popup = msg,
    fill = True,
    fill_color = icon_color
        ).add_to(map)

map